## Load Libraries

In [1]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import re
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle
import pickle
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

import matplotlib.font_manager
from matplotlib import style
style.use('seaborn') or plt.style.use('seaborn')

from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
import plotly.graph_objects as go
import seaborn as sns

from collections import Counter
import operator


from xgboost import plot_importance


from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

from sklearn.decomposition import FactorAnalysis

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import scipy.stats as ss


from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet
import math

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import scipy.stats as ss

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle


import scipy
from sklearn import metrics


from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [2]:
import my_bib as mb

## Define functions

In [3]:
def get_color(var, X_cat_, color_dict_):

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    index_code = X_name.index(var)
    cat = X_factors[index_code]

    return color_dict_[cat]

color_dict = {}
color_dict['Country'] = '#484849'
color_dict['HF'] = '#A31300'
color_dict['LSF'] = '#FF9E0D'
color_dict['PSF'] = '#FF4800'
color_dict['SDHF'] = '#1C4F9E'
color_dict['DF'] = '#009E32'

def get_bar_colors(data_, X_cat_):
    color_dict = {}
    color_dict['Country'] = '#484849'
    color_dict['HF'] = '#A31300'
    color_dict['LSF'] = '#FF9E0D'
    color_dict['PSF'] = '#FF4800'
    color_dict['SDHF'] = '#1C4F9E'
    color_dict['DF'] = '#009E32'
    
    #color_dict = {}
    #color_dict = {'DF':'#5975a4', 'MF':'#cc8963', 'SF':'#5f9e6e', 'SLF':'#b55d60',
    #              'n':'#857aab', 'country':'#8d7866'}#, '#d095bf'}

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    bar_color_total = []
    for i in list(data_.Features):
        if(i == 'Country'):
            bar_color_total.append(color_dict['Country'])
            continue
        index_code = X_name.index(i)
        cat = X_factors[index_code]
        bar_color_total.append(color_dict[cat])
    return bar_color_total



def plot_estimate_value(regression_model, X_cat_ = [], title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 14, pvalue_type = 'False'):

    df = regression_model[0]
    df.index.name = 'Features'
    df = df.iloc[1:-3, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()
    data = df.sort_values('Estimate', ascending=False)

    
    if(len(X_cat_)>0):
        
        bar_color = get_bar_colors(data, X_cat_)
        
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, palette =bar_color)
        plt.xlim(xlim)
    else:
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, color = 'darkblue')
        plt.xlim(xlim)


    y_step=0  
    for i in range(df.shape[0]):
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'gray'
        else:
            color = 'red'        
        
        if(pvalue_type == 'color'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]-0.005, y_step, 
                                 '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],2)),
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'white',
                                 bbox=dict(boxstyle="round",
                                           ec=color,
                                            fc=color,
                                           )
                                 )
                y_step+=1

        elif(pvalue_type == 'value'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                 '(' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],8))+')',
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'black',

                                 )
                y_step+=1
        else:
            if(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.01):
                
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '**',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )
            elif(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]>= 0.01 and df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.05):
                  
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '*',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )  
            else:
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )      
            y_step+=1
            
         
    text_diff =xlim[1]/2.2
    plt.text(xlim[1] - text_diff, df.shape[0]-1.5,r'$ R^2 $(' + str(np.round(regression_model[1],2)) + ') \t$F^2 $(' + str(np.round(regression_model[3],2))+ ') \n$MSE $(' + str(np.round(regression_model[4],2))+ ') \t$RMSE $(' + str(np.round(regression_model[5],2)) + ')',
                             size= 12, rotation=0.,
                             ha="left", va="center", color = 'black',
                             bbox=dict(boxstyle="round",
                                       ec='gray',
                                        fc='gray',
                                       )
                             )


    plt.locator_params(axis='x', nbins=4)


    
def plot_estimate_value_no_sort(regression_model, title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 14, ylabel = True, ylabelR = False ):

    df = regression_model
    df.index.name = 'Features'
    df = df.iloc[1:-3, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()

    #plt.figure(figsize=(8,12))
    plt.title(title)
    sns.barplot(x="Estimate", y="Features", data = df, color = 'Brown')
    if(ylabel == False):
        plt.ylabel('')
        plt.yticks([])
    plt.xlim(xlim)
    
    if(ylabelR):
        plt.tick_params (axis = 'y', which = 'both', labelleft = False, labelright = True)

    y_step=0  
    for i in range(df.shape[0]):
        
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'yellow'
        else:
            color = 'red'   
            
        plt.text(df['Estimate'].iloc[y_step]+0.015, y_step, 
                         '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],5)),
                         size= size, rotation=0.,
                         ha="left", va="center", color = color,
                         bbox=dict(boxstyle="round",
                                   ec=(50/255, 50/255, 50/255),
                                    fc=(50/255, 50/255, 50/255),
                                   )
                         )
        y_step+=1
        

        
def conf_interval(database):
    r_squared_list = []

    y_database = database['Barthel']
    X_database = database.drop('Barthel', axis=1)

    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
            Ridge(),
            {
                'alpha': ( 0.0001, 0.01, 0.001),
                'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                'max_iter': (1000, 10000, 100000, 1000000),
            },
            n_iter=10,
            random_state=i, 
            scoring='r2',
            cv=3
        )

        opt_Ridge.fit(X_train, y_train)

        r_squared_list.append(opt_Ridge.score(X_test, y_test))


    print(r'99% confidence interval (+-', np.round(np.std(r_squared_list)*2.58, 4),')')
    return np.round(np.std(r_squared_list)*2.58, 4)
        

def adj_r2_score_and_r2_score(clf, X, y):
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = r2_score(y, clf.predict(X))
    return [1 - (1 - r_squared) * ((n - 1) / (n - p - 1)), r_squared]


def mse(clf, X, y):
    return mean_squared_error(y, clf.predict(X))

def rmse(clf, X, y):
    mse = mean_squared_error(y, clf.predict(X))
    return math.sqrt(mse)    

def coef_se(clf, X, y):
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)

def coef_tval(clf, X, y):
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_tval_XGB_tree(clf, X, y):
    a = np.nan
    b = np.array(clf.feature_importances_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_pval(clf, X, y):

    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def coef_pval_XGB_tree(clf, X, y):

    n = X.shape[0]
    t = coef_tval_XGB_tree(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def residuals(clf, X, y, r_type='standardized'):

    # Make sure value of parameter 'r_type' is one we recognize
    assert r_type in ('raw', 'standardized', 'studentized'), (
        "Invalid option for 'r_type': {0}".format(r_type))
    y_true = y.view(dtype='float')
    # Use classifier to make predictions
    y_pred = clf.predict(X)
    # Make sure dimensions agree (Numpy still allows subtraction if they don't)
    assert y_true.shape == y_pred.shape, (
        "Dimensions of y_true {0} do not match y_pred {1}".format(y_true.shape,
                                                                  y_pred.shape))
    # Get raw residuals, or standardized or standardized residuals
    resids = y_pred - y_true
    if r_type == 'standardized':
        resids = resids / np.std(resids)
    elif r_type == 'studentized':
        # Prepare a blank array to hold studentized residuals
        studentized_resids = np.zeros(y_true.shape[0], dtype='float')
        # Calcluate hat matrix of X values so you can get leverage scores
        hat_matrix = np.dot(
            np.dot(X, np.linalg.inv(np.dot(np.transpose(X), X))),
            np.transpose(X))
        # For each point, calculate studentized residuals w/ leave-one-out MSE
        for i in range(y_true.shape[0]):
            # Make a mask so you can calculate leave-one-out MSE
            mask = np.ones(y_true.shape[0], dtype='bool')
            mask[i] = 0
            loo_mse = np.average(resids[mask] ** 2, axis=0)  # Leave-one-out MSE
            # Calculate studentized residuals
            studentized_resids[i] = resids[i] / np.sqrt(
                loo_mse * (1 - hat_matrix[i, i]))
        resids = studentized_resids
    return resids


def f_squared(clf, X, y):

    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return r_squared  / (1 - r_squared)



def summary(clf, X, y, xlabels=None, regressor = ''):

    print('Resumen del regresor ' + regressor + '\n')
    
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')

    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate','t value', 'p value']
    )
    
    if(regressor == 'XGBRegressor'):
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_[0]]), 6), np.round((clf.coef_), 6)))
        #coef_df['MSE'] = np.round(mse(clf, X, y), 6)
        #coef_df['RMSE'] = np.round(rmse(clf, X, y), 6)
        coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval(clf, X, y), 20)
        # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
            'Min': pd.Series(np.round(resids.min(), 4)),
            '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
            'Median': pd.Series(np.round(np.median(resids), 4)),
            '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
            'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
        # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)
        
        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
           r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
            f_sq))
    elif(regressor == 'XGBRegressorNoLinear'):
        coef_df = pd.DataFrame(
            index=['_intercept'] + list(xlabels),
            columns=['Estimate','t value', 'p value']
        )

        coef_df['Estimate'] = np.concatenate(
                (np.round(np.array([np.nan]), 6), np.round((clf.feature_importances_), 6)))

        coef_df['t value'] = np.round(coef_tval_XGB_tree(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval_XGB_tree(clf, X, y), 20)
            # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
                'Min': pd.Series(np.round(resids.min(), 4)),
                '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
                'Median': pd.Series(np.round(np.median(resids), 4)),
                '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
                'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
            # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)

        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
               r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
                f_sq))
    else:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
        #coef_df['MSE'] = np.round(mse(clf, X, y), 6)
        #coef_df['RMSE'] = np.round(rmse(clf, X, y), 6)
        coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval(clf, X, y), 20)
        # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
            'Min': pd.Series(np.round(resids.min(), 4)),
            '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
            'Median': pd.Series(np.round(np.median(resids), 4)),
            '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
            'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
        # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)
        
        mse_ = np.round(mse(clf, X, y), 6)
        rmse_ = np.round(rmse(clf, X, y), 6)
        
        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
           r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
            f_sq))
        print('MSE:  {0:.5f}'.format(
            mse_))        
        print('RMSE:  {0:.5f}'.format(
            rmse_))        
    print('---------------------------------------------------------------------------\n\n')
    
    
    
    empty_str = []
    for i in range(coef_df.shape[0]):
        empty_str.append('')
    
    coef_df['value'] = empty_str
    
    coef_df = coef_df.T
    coef_df['R-squared'] = ['','','', r_sq]
    coef_df['Adjusted R-squared'] = ['','','', r_sq_adj]
    coef_df['F-squared'] = ['','','', f_sq]
    #coef_df['MSE'] = ['','','', mse_]
    #coef_df['RMSE'] = ['','','', rmse_]
    return [coef_df.T, r_sq, r_sq_adj, f_sq, mse_, rmse_]

## Load Data

In [4]:
sabe_chile = pd.read_excel('../Data/brazil/SABE_chile.xlsx')
sabe_chile = sabe_chile.iloc[:,1::]

sabe_uruguay = pd.read_excel('../Data/brazil/SABE_uruguay.xlsx')
sabe_uruguay = sabe_uruguay.iloc[:,1::]

sabe_ecuador = pd.read_excel('../Data/brazil/SABE_ecuador.xlsx')
sabe_ecuador = sabe_ecuador.iloc[:,1::]

sabe_colombia = pd.read_excel('../Data/brazil/SABE_colombia.xlsx')
sabe_colombia = sabe_colombia.iloc[:,1::]

sabe_brazil = pd.read_excel('../Data/brazil/SABE_brazil_full.xlsx')
sabe_brazil = sabe_brazil.iloc[:,1::]

/home/marcoantonio/miniconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Add colors for future use, remove non-common features, order all df vars in same order

In [5]:
X_chile_01 = sabe_chile.copy()
X_uruguay_01 = sabe_uruguay.copy()
X_ecuador_01 = sabe_ecuador.copy()
X_colombia_01 = sabe_colombia.copy()
X_brazil_01 = sabe_brazil.copy()

X_cat = pd.read_csv('../Data/brazil/var_name_color_code_new.csv', encoding='latin-1', sep=";")

In [6]:
order_col = list(X_chile_01.columns)
order_col.remove('FS_Adversidad_AgresionVerb_j28')
order_col.remove('FS_Adversidad_AgresionFis_j27')
order_col.remove('FM_CardioMetab_IMC_none')
order_col.remove('FS_EstSocEcon_ElectricEquipment_none')
order_col.remove('FM_CardioMetab_ACV_c09')

X_chile_01     =  X_chile_01[order_col]
X_uruguay_01   =  X_uruguay_01[order_col]
X_ecuador_01   =  X_ecuador_01[order_col]
X_colombia_01  =  X_colombia_01[order_col]
X_brazil_01  = X_brazil_01[order_col]

X_chile_01.columns =  X_cat.newname
X_uruguay_01.columns =  X_cat.newname
X_ecuador_01.columns =  X_cat.newname
X_colombia_01.columns =  X_cat.newname
X_brazil_01.columns = X_cat.newname

In [7]:
print(X_chile_01.shape, '\n', X_uruguay_01.shape, '\n', X_ecuador_01.shape, '\n', 
      X_colombia_01.shape, '\n', X_brazil_01.shape)

(1301, 15) 
 (1450, 15) 
 (5235, 15) 
 (23694, 15) 
 (9412, 15)


In [8]:
#Sanity check
for i in range(len(order_col)):
    print('', X_chile_01.columns[i], '\n', X_uruguay_01.columns[i], '\n', X_ecuador_01.columns[i], '\n', 
          X_colombia_01.columns[i], '\n', X_brazil_01.columns[i])
    print('-------------------------------------------')

 Age 
 Age 
 Age 
 Age 
 Age
-------------------------------------------
 Sex 
 Sex 
 Sex 
 Sex 
 Sex
-------------------------------------------
 Diabetes 
 Diabetes 
 Diabetes 
 Diabetes 
 Diabetes
-------------------------------------------
 Education 
 Education 
 Education 
 Education 
 Education
-------------------------------------------
 Live Alone 
 Live Alone 
 Live Alone 
 Live Alone 
 Live Alone
-------------------------------------------
 House Condition 
 House Condition 
 House Condition 
 House Condition 
 House Condition
-------------------------------------------
 Hypertension 
 Hypertension 
 Hypertension 
 Hypertension 
 Hypertension
-------------------------------------------
 Heart Disease 
 Heart Disease 
 Heart Disease 
 Heart Disease 
 Heart Disease
-------------------------------------------
 Alcohol consumption 
 Alcohol consumption 
 Alcohol consumption 
 Alcohol consumption 
 Alcohol consumption
-------------------------------------------
 Physical activity

In [9]:
parameter_dict = {}

## Prepare data

### Chile

In [10]:
X_chile_n_IMC_ACV = X_chile_01.drop(['MMSE' ], axis=1)

In [11]:
X_chile_n_IMC_ACV.dropna(inplace=True)
print(X_chile_n_IMC_ACV.shape, X_chile_01.shape, X_chile_01.shape[0] - X_chile_n_IMC_ACV.shape[0])

(1057, 14) (1301, 15) 244


In [12]:
y_chile = X_chile_n_IMC_ACV['Barthel']
X_chile = X_chile_n_IMC_ACV.drop('Barthel', axis=1)

### Uruguay

In [13]:
X_uruguay_n_IMC_ACV = X_uruguay_01.drop(['MMSE' ], axis=1)

In [14]:
X_uruguay_n_IMC_ACV.dropna(inplace=True)
print(X_uruguay_n_IMC_ACV.shape, X_uruguay_01.shape, X_uruguay_01.shape[0] - X_uruguay_n_IMC_ACV.shape[0])

(1127, 14) (1450, 15) 323


In [15]:
y_uruguay = X_uruguay_n_IMC_ACV['Barthel']
X_uruguay = X_uruguay_n_IMC_ACV.drop('Barthel', axis=1)

### Ecuador

In [16]:
X_ecuador_n_IMC_ACV = X_ecuador_01.drop(['MMSE' ], axis=1)

In [17]:
X_ecuador_n_IMC_ACV.dropna(inplace=True)
print(X_ecuador_n_IMC_ACV.shape, X_ecuador_01.shape, X_ecuador_01.shape[0] - X_ecuador_n_IMC_ACV.shape[0])

(3059, 14) (5235, 15) 2176


In [18]:
y_ecuador = X_ecuador_n_IMC_ACV['Barthel']
X_ecuador = X_ecuador_n_IMC_ACV.drop('Barthel', axis=1)

### Colombia

In [19]:
X_colombia_n_IMC_ACV = X_colombia_01.drop(['MMSE' ], axis=1)

In [20]:
X_colombia_n_IMC_ACV.dropna(inplace=True)
print(X_colombia_n_IMC_ACV.shape, X_colombia_01.shape, X_colombia_01.shape[0] - X_colombia_n_IMC_ACV.shape[0])

(22748, 14) (23694, 15) 946


In [21]:
y_colombia = X_colombia_n_IMC_ACV['Barthel']
X_colombia = X_colombia_n_IMC_ACV.drop('Barthel', axis=1)

### Brazil

In [22]:
X_brazil_n_IMC_ACV = X_brazil_01.drop(['MMSE' ], axis=1)

In [23]:
X_brazil_n_IMC_ACV.dropna(inplace=True)
print(X_brazil_n_IMC_ACV.shape, X_brazil_01.shape, X_brazil_01.shape[0] - X_brazil_n_IMC_ACV.shape[0])

(7785, 14) (9412, 15) 1627


In [24]:
y_brazil = X_brazil_n_IMC_ACV['Barthel']
X_brazil = X_brazil_n_IMC_ACV.drop('Barthel', axis=1)

## Results

### HIC: Chile and Uruguay joint data

In [25]:
X_chile_n_IMC_ACV_colEcua = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua = X_uruguay_n_IMC_ACV.copy()

In [26]:
X_chile_n_IMC_ACV_colEcua['Country'] = np.ones([X_chile_n_IMC_ACV_colEcua.shape[0]])
X_uruguay_n_IMC_ACV_colEcua['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua.shape[0]])*2

In [27]:
X_chile_uruguay_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua, X_uruguay_n_IMC_ACV_colEcua], axis=0,))

In [28]:
y_chile_uruguay = X_chile_uruguay_n_IMC_ACV['Barthel']
X_chile_uruguay = X_chile_uruguay_n_IMC_ACV.drop('Barthel', axis=1)

In [29]:
print(X_chile_uruguay.shape)

(2184, 14)


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_chile_uruguay, y_chile_uruguay, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')



Ridge
val. score: 0.1790604302396498
test score: 0.2095046865467335
best parameters: OrderedDict([('alpha', 0.001), ('max_iter', 100000), ('solver', 'saga')])
---------------------------------------------



In [31]:
modelRidge_chile_uruguay = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_chile_uruguay.fit(X_chile_uruguay, y_chile_uruguay)

parameter_dict['chile_uruguay1'] = opt_Ridge.best_params_

modelRidge_sum_chile_uruguay = summary(modelRidge_chile_uruguay, X_chile_uruguay, y_chile_uruguay, X_chile_uruguay.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-2.0298 -0.5789 -0.1462 0.3562 4.4825


Coefficients:
                     Estimate  t value       p value
_intercept           0.107755   0.2619  7.934013e-01
Age                 -0.036995 -61.4186  0.000000e+00
Sex                  0.040799   0.9534  3.404854e-01
Diabetes             0.161649   2.9894  2.826782e-03
Education            0.042708   1.1142  2.653163e-01
Live Alone           0.101821   2.0173  4.378549e-02
House Condition      0.142912   7.0888  1.819433e-12
Hypertension         0.082325   2.1820  2.921951e-02
Heart Disease        0.203505   4.8854  1.107103e-06
Alcohol consumption  0.073291   3.8103  1.426596e-04
Physical activity   -0.426915  -8.9426  0.000000e+00
Smoking status      -0.009723  -0.2448  8.066380e-01
Falls                0.264693   6.5825  5.774092e-11
Mental Problems      0.269397   5.7347  1.112598e-08
Country             -0.044559  -1.1768  2.393956e-01
---
R-squared:  0.209

### HIC: Chile, Uruguay and Brazil joint data


In [32]:
X_chile_n_IMC_ACV_colEcua = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua = X_uruguay_n_IMC_ACV.copy()
X_brazil_n_IMC_ACV_colEcua = X_brazil_n_IMC_ACV.copy()

In [33]:
X_chile_n_IMC_ACV_colEcua['Country'] = np.ones([X_chile_n_IMC_ACV_colEcua.shape[0]])
X_uruguay_n_IMC_ACV_colEcua['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua.shape[0]])
X_brazil_n_IMC_ACV_colEcua['Country'] = np.ones([X_brazil_n_IMC_ACV_colEcua.shape[0]])*2

In [34]:
X_chi_uru_br_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua, X_uruguay_n_IMC_ACV_colEcua, X_brazil_n_IMC_ACV_colEcua], axis=0,))

In [35]:
y_chile_uruguay_br = X_chi_uru_br_n_IMC_ACV['Barthel']
X_chile_uruguay_br = X_chi_uru_br_n_IMC_ACV.drop('Barthel', axis=1)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_chile_uruguay_br, y_chile_uruguay_br, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')



Ridge
val. score: 0.1469110633733993
test score: 0.15303300734981318
best parameters: OrderedDict([('alpha', 0.001), ('max_iter', 100000), ('solver', 'sag')])
---------------------------------------------



In [37]:
modelRidge_chile_uruguay_br = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_chile_uruguay_br.fit(X_chile_uruguay_br, y_chile_uruguay_br)

parameter_dict['chile_uruguay_br'] = opt_Ridge.best_params_

modelRidge_sum_chile_uruguay_br = summary(modelRidge_chile_uruguay_br, X_chile_uruguay_br, y_chile_uruguay_br, X_chile_uruguay_br.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-1.8771 -0.4344 -0.1566 0.1755 6.1725


Coefficients:
                     Estimate  t value       p value
_intercept          -0.484206  -3.4908  4.836013e-04
Age                 -0.013999 -75.1528  0.000000e+00
Sex                  0.029027   1.8131  6.984476e-02
Diabetes             0.120257   5.8332  5.606642e-09
Education            0.050298   3.6017  3.176576e-04
Live Alone          -0.050133  -2.3654  1.802968e-02
House Condition      0.011955   4.8981  9.830215e-07
Hypertension         0.062902   4.0342  5.519749e-05
Heart Disease        0.206211   9.3763  0.000000e+00
Alcohol consumption  0.067021   5.2492  1.559254e-07
Physical activity   -0.210378 -13.0957  0.000000e+00
Smoking status      -0.054197  -3.3319  8.657442e-04
Falls                0.268002  15.1490  0.000000e+00
Mental Problems      0.335631  19.5686  0.000000e+00
Country             -0.000898  -0.0219  9.824885e-01
---
R-squared:  0.152

### LMIC: Ecuador, Colombia and Brazil joint data

In [38]:
X_ecuador_n_IMC_ACV_colEcua = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua = X_colombia_n_IMC_ACV.copy()
X_brazil_n_IMC_ACV_colEcua = X_brazil_n_IMC_ACV.copy()

In [39]:
X_ecuador_n_IMC_ACV_colEcua['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])
X_colombia_n_IMC_ACV_colEcua['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])
X_brazil_n_IMC_ACV_colEcua['Country'] = np.ones([X_brazil_n_IMC_ACV_colEcua.shape[0]])*2

In [40]:
X_ecu_col_br_n_IMC_ACV = shuffle(pd.concat([X_ecuador_n_IMC_ACV_colEcua, X_colombia_n_IMC_ACV_colEcua, X_brazil_n_IMC_ACV_colEcua], axis=0,))

In [41]:
y_ecu_col_br = X_ecu_col_br_n_IMC_ACV['Barthel']
X_ecu_col_br = X_ecu_col_br_n_IMC_ACV.drop('Barthel', axis=1)

In [42]:
print(X_ecu_col_br.shape)

(33592, 14)


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_ecu_col_br, y_ecu_col_br, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


Ridge
val. score: 0.2614709675426104
test score: 0.2599497472464367
best parameters: OrderedDict([('alpha', 0.001), ('max_iter', 10000), ('solver', 'sag')])
---------------------------------------------



In [44]:
modelRidge_ecu_col_br = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_ecu_col_br.fit(X_ecu_col_br, y_ecu_col_br)

parameter_dict['ecu_col_br'] = opt_Ridge.best_params_

modelRidge_sum_ecu_col_br = summary(modelRidge_ecu_col_br, X_ecu_col_br, y_ecu_col_br, X_ecu_col_br.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q   Max
-1.9137 -0.4738 -0.1604 0.2601 6.332


Coefficients:
                     Estimate   t value       p value
_intercept           1.895477   23.6442  0.000000e+00
Age                 -0.038158 -359.2723  0.000000e+00
Sex                  0.072391    7.6447  2.153833e-14
Diabetes             0.098649    8.3277  2.220400e-16
Education            0.036572    3.7371  1.864308e-04
Live Alone           0.048778    4.4225  9.788597e-06
House Condition      0.014638    8.6427  0.000000e+00
Hypertension         0.055165    6.0530  1.436647e-09
Heart Disease        0.163966   12.6390  0.000000e+00
Alcohol consumption  0.074560    7.4589  8.926193e-14
Physical activity   -0.477698  -46.0968  0.000000e+00
Smoking status      -0.048748   -5.8229  5.837666e-09
Falls                0.211818   21.9579  0.000000e+00
Mental Problems      0.368162   28.7832  0.000000e+00
Country             -0.259728  -13.0304  0.000000e+00
---
R-s

### LMIC: Ecuador and Colombia joint data

In [45]:
X_ecuador_n_IMC_ACV_colEcua = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua = X_colombia_n_IMC_ACV.copy()

In [46]:
X_ecuador_n_IMC_ACV_colEcua['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])
X_colombia_n_IMC_ACV_colEcua['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])*2

In [47]:
X_ecuador_colombia_n_IMC_ACV = shuffle(pd.concat([X_ecuador_n_IMC_ACV_colEcua, X_colombia_n_IMC_ACV_colEcua], axis=0,))

In [48]:
y_ecuador_colombia = X_ecuador_colombia_n_IMC_ACV['Barthel']
X_ecuador_colombia = X_ecuador_colombia_n_IMC_ACV.drop('Barthel', axis=1)

In [49]:
print(X_ecuador_colombia.shape)

(25807, 14)


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X_ecuador_colombia, y_ecuador_colombia, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


Ridge
val. score: 0.33100582647729965
test score: 0.3206400654874393
best parameters: OrderedDict([('alpha', 0.001), ('max_iter', 1000), ('solver', 'sag')])
---------------------------------------------



In [51]:
modelRidge_ecuador_colombia = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_ecuador_colombia.fit(X_ecuador_colombia, y_ecuador_colombia)

parameter_dict['ecuador_colombia'] = opt_Ridge.best_params_

modelRidge_sum_ecuador_colombia = summary(modelRidge_ecuador_colombia, X_ecuador_colombia, y_ecuador_colombia, X_ecuador_colombia.columns, 'Ridge')


Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q    Max
-2.2664 -0.4801 -0.1339 0.2564 5.0564


Coefficients:
                     Estimate   t value       p value
_intercept           2.211347   22.4509  0.000000e+00
Age                 -0.049552 -392.7641  0.000000e+00
Sex                  0.088169    7.9680  1.554310e-15
Diabetes             0.116838    8.6214  0.000000e+00
Education            0.041313    3.2998  9.688483e-04
Live Alone           0.172976   11.4751  0.000000e+00
House Condition      0.019549    9.1167  0.000000e+00
Hypertension         0.058939    5.6662  1.475274e-08
Heart Disease        0.149358   10.2895  0.000000e+00
Alcohol consumption  0.058988    5.0231  5.118566e-07
Physical activity   -0.616833  -48.6597  0.000000e+00
Smoking status      -0.033912   -3.2663  1.091178e-03
Falls                0.193746   17.9930  0.000000e+00
Mental Problems      0.366240   21.4448  0.000000e+00
Country              0.154259    8.5658  0.000000e+00
---
R

### Brazil

In [52]:
X_brazil_n_IMC_ACV_colEcua = X_brazil_n_IMC_ACV.copy()

# X_brazil_n_IMC_ACV_colEcua['Country'] = np.ones([X_brazil_n_IMC_ACV_colEcua.shape[0]])

X_br_n_IMC_ACV = shuffle(pd.concat([X_brazil_n_IMC_ACV_colEcua], axis=0,))

y_br = X_br_n_IMC_ACV['Barthel']
X_br = X_br_n_IMC_ACV.drop('Barthel', axis=1)

print(X_br.shape)

(7785, 13)


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_br, y_br, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')

Ridge
val. score: 0.15079172416484402
test score: 0.14146763919295136
best parameters: OrderedDict([('alpha', 0.01), ('max_iter', 100000), ('solver', 'sag')])
---------------------------------------------



In [54]:
modelRidge_br = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_br.fit(X_br, y_br)

parameter_dict['br'] = opt_Ridge.best_params_

modelRidge_sum_br = summary(modelRidge_br, X_br, y_br, X_br.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
   Min      1Q  Median     3Q    Max
-1.406 -0.3909 -0.1587 0.1239 6.1479


Coefficients:
                     Estimate  t value       p value
_intercept          -0.815307  -6.6861  2.451062e-11
Age                 -0.010021 -51.4258  0.000000e+00
Sex                  0.026028   1.5293  1.262263e-01
Diabetes             0.119388   5.5141  3.617260e-08
Education            0.060849   4.1331  3.616536e-05
Live Alone          -0.057578  -2.4730  1.341895e-02
House Condition      0.009947   3.6228  2.933432e-04
Hypertension         0.068636   4.1023  4.132059e-05
Heart Disease        0.207526   8.0602  8.881800e-16
Alcohol consumption  0.070901   2.5318  1.136711e-02
Physical activity   -0.174036 -10.4470  0.000000e+00
Smoking status      -0.034890  -1.5616  1.184208e-01
Falls                0.260127  13.3992  0.000000e+00
Mental Problems      0.358074  19.9581  0.000000e+00
---
R-squared:  0.15288,    Adjusted R-squared:  0.15146
F-squared:  0.18047

### All LAC1: Chile, Uruguay, Ecuador, Colombia, Brazil

In [55]:
X_chile_n_IMC_ACV_colEcua_01 = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua_01 = X_uruguay_n_IMC_ACV.copy()
X_ecuador_n_IMC_ACV_colEcua_01 = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua_01 = X_colombia_n_IMC_ACV.copy()
X_brazil_n_IMC_ACV_colEcua_01 = X_brazil_n_IMC_ACV.copy()

In [56]:
X_chile_n_IMC_ACV_colEcua_01['Country']   = np.ones([X_chile_n_IMC_ACV_colEcua_01.shape[0]])
X_uruguay_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua_01.shape[0]])
X_ecuador_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])*2
X_colombia_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])*2
X_brazil_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_brazil_n_IMC_ACV_colEcua_01.shape[0]])*2

In [57]:
X_all_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua_01, X_uruguay_n_IMC_ACV_colEcua_01, X_ecuador_n_IMC_ACV_colEcua_01, X_colombia_n_IMC_ACV_colEcua_01, X_brazil_n_IMC_ACV_colEcua_01], axis=0,))

In [58]:
y_all = X_all_n_IMC_ACV['Barthel']
X_all = X_all_n_IMC_ACV.drop('Barthel', axis=1)

In [59]:
print(X_all.shape)

(35776, 14)


In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


Ridge
val. score: 0.25381093720020426
test score: 0.25683027423149873
best parameters: OrderedDict([('alpha', 0.001), ('max_iter', 1000000), ('solver', 'saga')])
---------------------------------------------



In [61]:
modelRidge_all = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_all.fit(X_all, y_all)

parameter_dict['all1'] = opt_Ridge.best_params_

modelRidge_sum_all = summary(modelRidge_all, X_all, y_all, X_all.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min     1Q  Median     3Q    Max
-1.9364 -0.485 -0.1644 0.2672 6.3544


Coefficients:
                     Estimate   t value       p value
_intercept           1.364116   18.8765  0.000000e+00
Age                 -0.036876 -347.2172  0.000000e+00
Sex                  0.079425    8.5688  0.000000e+00
Diabetes             0.101981    8.8319  0.000000e+00
Education            0.019564    2.0948  3.619397e-02
Live Alone           0.000439    0.0443  9.646551e-01
House Condition      0.020594   12.6262  0.000000e+00
Hypertension         0.059583    6.7202  1.841438e-11
Heart Disease        0.162963   13.1667  0.000000e+00
Alcohol consumption  0.122735   15.3798  0.000000e+00
Physical activity   -0.463234  -45.6039  0.000000e+00
Smoking status       0.015121    2.0268  4.269069e-02
Falls                0.211744   22.6385  0.000000e+00
Mental Problems      0.377749   30.4409  0.000000e+00
Country             -0.006055   -0.3463  7.291202e-01
---
R-s

### All LAC2: Chile, Uruguay, Ecuador, Colombia

In [62]:
X_chile_n_IMC_ACV_colEcua_01 = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua_01 = X_uruguay_n_IMC_ACV.copy()
X_ecuador_n_IMC_ACV_colEcua_01 = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua_01 = X_colombia_n_IMC_ACV.copy()

In [63]:
X_chile_n_IMC_ACV_colEcua_01['Country']   = np.ones([X_chile_n_IMC_ACV_colEcua_01.shape[0]])
X_uruguay_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua_01.shape[0]])
X_ecuador_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])*2
X_colombia_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])*2

In [64]:
X_all_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua_01, X_uruguay_n_IMC_ACV_colEcua_01, X_ecuador_n_IMC_ACV_colEcua_01, X_colombia_n_IMC_ACV_colEcua_01], axis=0,))

In [65]:
y_all = X_all_n_IMC_ACV['Barthel']
X_all = X_all_n_IMC_ACV.drop('Barthel', axis=1)

In [66]:
print(X_all.shape)

(27991, 14)


In [67]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


Ridge
val. score: 0.32248476740256193
test score: 0.2989719492959598
best parameters: OrderedDict([('alpha', 0.001), ('max_iter', 10000), ('solver', 'sparse_cg')])
---------------------------------------------



In [68]:
modelRidge_all2 = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_all2.fit(X_all, y_all)

parameter_dict['all2'] = opt_Ridge.best_params_

modelRidge_sum_all2 = summary(modelRidge_all2, X_all, y_all, X_all.columns, 'Ridge')

Resumen del regresor Ridge

Residuals:
    Min      1Q  Median     3Q   Max
-2.1505 -0.4871 -0.1382 0.2632 5.055


Coefficients:
                     Estimate   t value       p value
_intercept           2.596479   29.2439  0.000000e+00
Age                 -0.048641 -396.5553  0.000000e+00
Sex                  0.082511    7.6818  1.620926e-14
Diabetes             0.119917    9.1271  0.000000e+00
Education            0.045347    3.8163  1.357458e-04
Live Alone           0.090410    7.2335  4.827250e-13
House Condition      0.017854    8.7330  0.000000e+00
Hypertension         0.060388    6.0182  1.785488e-09
Heart Disease        0.155054   11.3016  0.000000e+00
Alcohol consumption  0.058317    6.0660  1.328338e-09
Physical activity   -0.600302  -49.4214  0.000000e+00
Smoking status      -0.034897   -3.4656  5.298640e-04
Falls                0.200346   19.2653  0.000000e+00
Mental Problems      0.359101   22.4625  0.000000e+00
Country             -0.044642   -2.4691  1.355181e-02
---
R-s

## T-test between HIC and LMIC Coefficients

In [134]:
def get_order_var(model_):

    df = model_[0]['Estimate'].iloc[1:-3]
    df = np.abs(pd.to_numeric(df,errors = 'coerce'))
    df = list(df.sort_values(ascending=False).index)
    return df

def beta_values_ttest(modelRidge_sum_model1, modelRidge_sum_model2, X_1, X_2, only_significance = False):
    col = get_order_var(modelRidge_sum_model1)
    df = modelRidge_sum_model1

    feature_sign = []

    for i in col:
        if df[0]['p value'][i] < 0.05:
            feature_sign.append(i)


    df = modelRidge_sum_model2

    for i in col:
        if df[0]['p value'][i] < 0.05 and i not in feature_sign:
            feature_sign.append(i)


    beta_dict = {}
    for i in feature_sign:

        beta_dict['HIC_' + i] = []
        beta_dict['LMIC_' + i] = []




    y_database = X_1['Barthel']
    X_database = X_1.drop('Barthel', axis=1)
    
    if(only_significance):
        X_database = X_database[feature_sign]



    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
                Ridge(),
                {
                    'alpha': ( 0.0001, 0.01, 0.001),
                    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                    'max_iter': (1000, 10000, 100000, 1000000),
                },
                n_iter=10,
                random_state=i, 
                scoring='r2',
                cv=3
            )

        opt_Ridge.fit(X_train, y_train)


        model = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                                 max_iter=opt_Ridge.best_params_['max_iter'])

        model.fit(X_test, y_test)

        coef_df = pd.DataFrame(
            index=['_intercept'] + list(X_test.columns),
            columns=['Estimate']
            )

        coef_df['Estimate'] = np.concatenate((np.round(np.array([model.intercept_]), 12), np.round((model.coef_), 12)))

        # save Coefficients HIC
        for j in feature_sign:
            beta_dict['HIC_' + j].append(coef_df['Estimate'][j])


    y_database = X_2['Barthel']
    X_database = X_2.drop('Barthel', axis=1)
    if(only_significance):
        X_database = X_database[feature_sign]

    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
                Ridge(),
                {
                    'alpha': ( 0.0001, 0.01, 0.001),
                    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                    'max_iter': (1000, 10000, 100000, 1000000),
                },
                n_iter=10,
                random_state=i, 
                scoring='r2',
                cv=3
            )

        opt_Ridge.fit(X_train, y_train)

        model = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                                 max_iter=opt_Ridge.best_params_['max_iter'])

        model.fit(X_test, y_test)

        coef_df = pd.DataFrame(
            index=['_intercept'] + list(X_test.columns),
            columns=['Estimate']
            )

        coef_df['Estimate'] = np.concatenate((np.round(np.array([model.intercept_]), 12), np.round((model.coef_), 12)))

        #Save coefficients LMIC
        for j in feature_sign:
            beta_dict['LMIC_'+ j].append(coef_df['Estimate'][j])

    beta_df = pd.DataFrame(beta_dict)


    from scipy import stats
    import statsmodels.api

    result_scipy_df = pd.DataFrame(
            index=['p value'] ,
            columns=feature_sign
            )

    result_stats_df = pd.DataFrame(
            index=['p value'] ,
            columns=feature_sign
            )

    for i in feature_sign:
        result_scipy_df[i] = stats.ttest_ind(beta_dict['HIC_'+ i], beta_dict['LMIC_' + i], equal_var=False)[1]
        result_stats_df[i] = statsmodels.stats.weightstats.ttest_ind(beta_dict['HIC_'+ i], beta_dict['LMIC_' + i], alternative="two-sided",usevar="unequal")[1]


    beta_df_stat = beta_df.describe().round(3).iloc[1:3,:]
    beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*1.95,ignore_index=True)
    beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*2.58,ignore_index=True)

    beta_df_stat = beta_df_stat.rename(index={0:'mean', 1:'std', 2:'95', 3:'99'})
    beta_df_stat.round(3)


    general_result_df = pd.DataFrame(
            index=feature_sign,
            columns=['HIC_mean', 'HIC_std','LMIC_mean','LMIC_std', 'ttest']
            )



    count = 0
    for i in feature_sign:
        general_result_df.iloc[count, 0] = np.round(beta_df_stat['HIC_' + i][0],2)
        general_result_df.iloc[count, 1] = np.round(beta_df_stat['HIC_' + i][1],2)

        general_result_df.iloc[count, 2] = np.round(beta_df_stat['LMIC_' + i][0],2)
        general_result_df.iloc[count, 3] = np.round(beta_df_stat['LMIC_' + i][1],2)

        general_result_df.iloc[count, 4] = np.round(result_stats_df[i][0],2)

        count +=1

    return general_result_df

In [135]:
Coef_t_test = beta_values_ttest(modelRidge_sum_chile_uruguay, modelRidge_sum_ecu_col_br, 
                  X_chile_uruguay_n_IMC_ACV, X_ecu_col_br_n_IMC_ACV, True)#.to_excel('./Cognition_beta_ttest_all_feat.xlsx')

/tmp/ipykernel_761862/3224612373.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*1.95,ignore_index=True)
/tmp/ipykernel_761862/3224612373.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*2.58,ignore_index=True)


In [136]:
Coef_t_test

,HIC_mean,HIC_std,LMIC_mean,LMIC_std,ttest
Physical activity,-0.44,0.06,-0.38,0.02,0.01
Mental Problems,0.22,0.1,0.02,0.02,0.0
Falls,0.28,0.12,0.13,0.02,0.0
Heart Disease,0.23,0.08,0.12,0.02,0.0
Diabetes,0.12,0.1,0.06,0.02,0.09
House Condition,0.17,0.05,0.02,0.0,0.0
Live Alone,0.09,0.1,0.11,0.01,0.58
Hypertension,0.09,0.06,0.06,0.02,0.15
Alcohol consumption,0.05,0.03,0.08,0.02,0.0
Age,-0.04,0.0,-0.04,0.0,0.02


## T-test between Brazil and LMIC Coefficients

In [140]:
def get_order_var(model_):

    df = model_[0]['Estimate'].iloc[1:-3]
    df = np.abs(pd.to_numeric(df,errors = 'coerce'))
    df = list(df.sort_values(ascending=False).index)
    return df

def beta_values_ttest(modelRidge_sum_model1, modelRidge_sum_model2, X_1, X_2, only_significance = False):
    col = get_order_var(modelRidge_sum_model1)
    df = modelRidge_sum_model1

    feature_sign = []

    for i in col:
        if df[0]['p value'][i] < 0.05:
            feature_sign.append(i)


    df = modelRidge_sum_model2

    for i in col:
        if df[0]['p value'][i] < 0.05 and i not in feature_sign:
            feature_sign.append(i)


    beta_dict = {}
    for i in feature_sign:

        beta_dict['Brazil_' + i] = []
        beta_dict['LMIC_' + i] = []




    y_database = X_1['Barthel']
    X_database = X_1.drop('Barthel', axis=1)
    
    if(only_significance):
        X_database = X_database[feature_sign]



    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
                Ridge(),
                {
                    'alpha': ( 0.0001, 0.01, 0.001),
                    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                    'max_iter': (1000, 10000, 100000, 1000000),
                },
                n_iter=10,
                random_state=i, 
                scoring='r2',
                cv=3
            )

        opt_Ridge.fit(X_train, y_train)


        model = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                                 max_iter=opt_Ridge.best_params_['max_iter'])

        model.fit(X_test, y_test)

        coef_df = pd.DataFrame(
            index=['_intercept'] + list(X_test.columns),
            columns=['Estimate']
            )

        coef_df['Estimate'] = np.concatenate((np.round(np.array([model.intercept_]), 12), np.round((model.coef_), 12)))

        # save Coefficients HIC
        for j in feature_sign:
            beta_dict['Brazil_' + j].append(coef_df['Estimate'][j])


    y_database = X_2['Barthel']
    X_database = X_2.drop('Barthel', axis=1)
    if(only_significance):
        X_database = X_database[feature_sign]

    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
                Ridge(),
                {
                    'alpha': ( 0.0001, 0.01, 0.001),
                    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                    'max_iter': (1000, 10000, 100000, 1000000),
                },
                n_iter=10,
                random_state=i, 
                scoring='r2',
                cv=3
            )

        opt_Ridge.fit(X_train, y_train)

        model = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                                 max_iter=opt_Ridge.best_params_['max_iter'])

        model.fit(X_test, y_test)

        coef_df = pd.DataFrame(
            index=['_intercept'] + list(X_test.columns),
            columns=['Estimate']
            )

        coef_df['Estimate'] = np.concatenate((np.round(np.array([model.intercept_]), 12), np.round((model.coef_), 12)))

        #Save coefficients LMIC
        for j in feature_sign:
            beta_dict['LMIC_'+ j].append(coef_df['Estimate'][j])

    beta_df = pd.DataFrame(beta_dict)


    from scipy import stats
    import statsmodels.api

    result_scipy_df = pd.DataFrame(
            index=['p value'] ,
            columns=feature_sign
            )

    result_stats_df = pd.DataFrame(
            index=['p value'] ,
            columns=feature_sign
            )

    for i in feature_sign:
        result_scipy_df[i] = stats.ttest_ind(beta_dict['Brazil_'+ i], beta_dict['LMIC_' + i], equal_var=False)[1]
        result_stats_df[i] = statsmodels.stats.weightstats.ttest_ind(beta_dict['Brazil_'+ i], beta_dict['LMIC_' + i], alternative="two-sided",usevar="unequal")[1]


    beta_df_stat = beta_df.describe().round(3).iloc[1:3,:]
    beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*1.95,ignore_index=True)
    beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*2.58,ignore_index=True)

    beta_df_stat = beta_df_stat.rename(index={0:'mean', 1:'std', 2:'95', 3:'99'})
    beta_df_stat.round(3)


    general_result_df = pd.DataFrame(
            index=feature_sign,
            columns=['Brazil_mean', 'Brazil_std','LMIC_mean','LMIC_std', 'ttest']
            )



    count = 0
    for i in feature_sign:
        general_result_df.iloc[count, 0] = np.round(beta_df_stat['Brazil_' + i][0],2)
        general_result_df.iloc[count, 1] = np.round(beta_df_stat['Brazil_' + i][1],2)

        general_result_df.iloc[count, 2] = np.round(beta_df_stat['LMIC_' + i][0],2)
        general_result_df.iloc[count, 3] = np.round(beta_df_stat['LMIC_' + i][1],2)

        general_result_df.iloc[count, 4] = np.round(result_stats_df[i][0],2)

        count +=1

    return general_result_df

In [141]:
Coef_t_test = beta_values_ttest(modelRidge_sum_br, modelRidge_sum_ecuador_colombia, 
                  X_br_n_IMC_ACV, X_ecuador_colombia_n_IMC_ACV, True)#.to_excel('./Cognition_beta_ttest_all_feat.xlsx')

Coef_t_test

/tmp/ipykernel_761862/3543944391.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*1.95,ignore_index=True)
/tmp/ipykernel_761862/3543944391.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*2.58,ignore_index=True)


,Brazil_mean,Brazil_std,LMIC_mean,LMIC_std,ttest
Mental Problems,-0.35,0.03,0.38,0.05,0.0
Falls,-0.26,0.04,0.2,0.02,0.0
Heart Disease,-0.19,0.1,0.15,0.02,0.0
Physical activity,0.18,0.02,-0.61,0.02,0.0
Diabetes,-0.12,0.05,0.12,0.02,0.0
Alcohol consumption,0.08,0.03,0.06,0.01,0.1
Hypertension,-0.06,0.03,0.05,0.01,0.0
Education,0.05,0.02,0.04,0.02,0.3
Live Alone,0.06,0.05,0.09,0.02,0.06
Age,-0.01,0.0,-0.05,0.0,0.0


## T-test between Brazil and HMIC Coefficients

In [142]:
def get_order_var(model_):

    df = model_[0]['Estimate'].iloc[1:-3]
    df = np.abs(pd.to_numeric(df,errors = 'coerce'))
    df = list(df.sort_values(ascending=False).index)
    return df

def beta_values_ttest(modelRidge_sum_model1, modelRidge_sum_model2, X_1, X_2, only_significance = False):
    col = get_order_var(modelRidge_sum_model1)
    df = modelRidge_sum_model1

    feature_sign = []

    for i in col:
        if df[0]['p value'][i] < 0.05:
            feature_sign.append(i)


    df = modelRidge_sum_model2

    for i in col:
        if df[0]['p value'][i] < 0.05 and i not in feature_sign:
            feature_sign.append(i)


    beta_dict = {}
    for i in feature_sign:

        beta_dict['HIC_' + i] = []
        beta_dict['Brazil_' + i] = []




    y_database = X_1['Barthel']
    X_database = X_1.drop('Barthel', axis=1)
    
    if(only_significance):
        X_database = X_database[feature_sign]



    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
                Ridge(),
                {
                    'alpha': ( 0.0001, 0.01, 0.001),
                    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                    'max_iter': (1000, 10000, 100000, 1000000),
                },
                n_iter=10,
                random_state=i, 
                scoring='r2',
                cv=3
            )

        opt_Ridge.fit(X_train, y_train)


        model = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                                 max_iter=opt_Ridge.best_params_['max_iter'])

        model.fit(X_test, y_test)

        coef_df = pd.DataFrame(
            index=['_intercept'] + list(X_test.columns),
            columns=['Estimate']
            )

        coef_df['Estimate'] = np.concatenate((np.round(np.array([model.intercept_]), 12), np.round((model.coef_), 12)))

        # save Coefficients HIC
        for j in feature_sign:
            beta_dict['HIC_' + j].append(coef_df['Estimate'][j])


    y_database = X_2['Barthel']
    X_database = X_2.drop('Barthel', axis=1)
    if(only_significance):
        X_database = X_database[feature_sign]

    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
                Ridge(),
                {
                    'alpha': ( 0.0001, 0.01, 0.001),
                    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                    'max_iter': (1000, 10000, 100000, 1000000),
                },
                n_iter=10,
                random_state=i, 
                scoring='r2',
                cv=3
            )

        opt_Ridge.fit(X_train, y_train)

        model = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                                 max_iter=opt_Ridge.best_params_['max_iter'])

        model.fit(X_test, y_test)

        coef_df = pd.DataFrame(
            index=['_intercept'] + list(X_test.columns),
            columns=['Estimate']
            )

        coef_df['Estimate'] = np.concatenate((np.round(np.array([model.intercept_]), 12), np.round((model.coef_), 12)))

        #Save coefficients LMIC
        for j in feature_sign:
            beta_dict['Brazil_'+ j].append(coef_df['Estimate'][j])

    beta_df = pd.DataFrame(beta_dict)


    from scipy import stats
    import statsmodels.api

    result_scipy_df = pd.DataFrame(
            index=['p value'] ,
            columns=feature_sign
            )

    result_stats_df = pd.DataFrame(
            index=['p value'] ,
            columns=feature_sign
            )

    for i in feature_sign:
        result_scipy_df[i] = stats.ttest_ind(beta_dict['HIC_'+ i], beta_dict['Brazil_' + i], equal_var=False)[1]
        result_stats_df[i] = statsmodels.stats.weightstats.ttest_ind(beta_dict['HIC_'+ i], beta_dict['Brazil_' + i], alternative="two-sided",usevar="unequal")[1]


    beta_df_stat = beta_df.describe().round(3).iloc[1:3,:]
    beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*1.95,ignore_index=True)
    beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*2.58,ignore_index=True)

    beta_df_stat = beta_df_stat.rename(index={0:'mean', 1:'std', 2:'95', 3:'99'})
    beta_df_stat.round(3)


    general_result_df = pd.DataFrame(
            index=feature_sign,
            columns=['HIC_mean', 'HIC_std','Brazil_mean','Brazil_std', 'ttest']
            )



    count = 0
    for i in feature_sign:
        general_result_df.iloc[count, 0] = np.round(beta_df_stat['HIC_' + i][0],2)
        general_result_df.iloc[count, 1] = np.round(beta_df_stat['HIC_' + i][1],2)

        general_result_df.iloc[count, 2] = np.round(beta_df_stat['Brazil_' + i][0],2)
        general_result_df.iloc[count, 3] = np.round(beta_df_stat['Brazil_' + i][1],2)

        general_result_df.iloc[count, 4] = np.round(result_stats_df[i][0],2)

        count +=1

    return general_result_df

In [143]:
Coef_t_test = beta_values_ttest(modelRidge_sum_br, modelRidge_sum_chile_uruguay, 
                  X_br_n_IMC_ACV, X_chile_uruguay_n_IMC_ACV, True)#.to_excel('./Cognition_beta_ttest_all_feat.xlsx')

Coef_t_test

/tmp/ipykernel_761862/1082424443.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*1.95,ignore_index=True)
/tmp/ipykernel_761862/1082424443.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*2.58,ignore_index=True)


,HIC_mean,HIC_std,Brazil_mean,Brazil_std,ttest
Mental Problems,-0.35,0.04,0.23,0.1,0.0
Falls,-0.26,0.04,0.28,0.11,0.0
Heart Disease,-0.19,0.1,0.22,0.08,0.0
Physical activity,0.18,0.02,-0.44,0.06,0.0
Diabetes,-0.12,0.05,0.12,0.1,0.0
Alcohol consumption,0.08,0.03,0.06,0.03,0.04
Hypertension,-0.06,0.03,0.09,0.06,0.0
Education,0.05,0.02,0.04,0.09,0.71
Live Alone,0.06,0.05,0.08,0.1,0.41
Age,-0.01,0.0,-0.04,0.0,0.0


## F-Statistic (HIC, LMIC, ALL LAC)

In [ ]:
#def f_stat(clf, X, y):
    """Calculate summary F-statistic for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic value.
    """
    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return (r_squared / p) / ((1 - r_squared) / (n - p - 1))


#def f_stat_pvalue(clf, X, y):
    """Calculate summary F-statistic p value for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic p value.
    """
    n = X.shape[0] # Esto se extrae par los grados de libertad del numeador y el denomindor (no. predictores, no. sujetos - no. predictores-1)
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    
    return np.round(scipy.stats.f.sf(f_stat(clf, X, y), n, (n - p - 1)), 15)

#def compute_f_statistics(clf, X, y):
    return [f_stat(clf, X, y), f_stat_pvalue(clf, X, y)]

#F_statistics = {}



#F_statistics['HIC'] = [compute_f_statistics(modelRidge_chile_uruguay, X_chile_uruguay, y_chile_uruguay)]
#F_statistics['LMIC'] = [compute_f_statistics(modelRidge_ecu_col_br, X_ecu_col_br, y_ecu_col_br)]
#F_statistics['ALL'] = [compute_f_statistics(modelRidge_all, X_all, y_all)]

#F_statistics_pd = pd.DataFrame(F_statistics, index = ['F-statisitcs, F-pvalue']).T
#F_statistics_pd

,"F-statisitcs, F-pvalue"
HIC,"[44.85070088314512, 0.0]"
LMIC,"[974.0282296529596, 0.0]"
ALL,"[866.7126497460733, 0.0]"


In [ ]:
# F_statistics_pd.to_csv("./res_files/Cross_cog/F_stats.csv")

## Summary Results

### HIC

In [69]:
#modelRidge_sum_chile_uruguay[0].to_csv('./res_files/Cross_cog/Ridge_HIC.csv')
modelRidge_sum_chile_uruguay[0]

,Estimate,t value,p value,value
_intercept,0.107755,0.2619,0.793401,
Age,-0.036995,-61.4186,0.0,
Sex,0.040799,0.9534,0.340485,
Diabetes,0.161649,2.9894,0.002827,
Education,0.042708,1.1142,0.265316,
Live Alone,0.101821,2.0173,0.043785,
House Condition,0.142912,7.0888,0.0,
Hypertension,0.082325,2.182,0.02922,
Heart Disease,0.203505,4.8854,0.000001,
Alcohol consumption,0.073291,3.8103,0.000143,


In [70]:
#modelRidge_sum_chile_uruguay_br[0].to_csv('./res_files/Cross_cog/Ridge_HIC.csv')
modelRidge_sum_chile_uruguay_br[0]

,Estimate,t value,p value,value
_intercept,-0.484206,-3.4908,0.000484,
Age,-0.013999,-75.1528,0.0,
Sex,0.029027,1.8131,0.069845,
Diabetes,0.120257,5.8332,0.0,
Education,0.050298,3.6017,0.000318,
Live Alone,-0.050133,-2.3654,0.01803,
House Condition,0.011955,4.8981,0.000001,
Hypertension,0.062902,4.0342,0.000055,
Heart Disease,0.206211,9.3763,0.0,
Alcohol consumption,0.067021,5.2492,0.0,


### LMIC

In [71]:
#modelRidge_sum_ecu_col_br[0].to_csv('./res_files/Cross_cog/Ridge_LMIC.csv')
modelRidge_sum_ecu_col_br[0]

,Estimate,t value,p value,value
_intercept,1.895477,23.6442,0.0,
Age,-0.038158,-359.2723,0.0,
Sex,0.072391,7.6447,0.0,
Diabetes,0.098649,8.3277,0.0,
Education,0.036572,3.7371,0.000186,
Live Alone,0.048778,4.4225,0.00001,
House Condition,0.014638,8.6427,0.0,
Hypertension,0.055165,6.053,0.0,
Heart Disease,0.163966,12.639,0.0,
Alcohol consumption,0.07456,7.4589,0.0,


In [72]:
#modelRidge_sum_ecu_col_br[0].to_csv('./res_files/Cross_cog/Ridge_LMIC.csv')
modelRidge_sum_ecuador_colombia[0]

,Estimate,t value,p value,value
_intercept,2.211347,22.4509,0.0,
Age,-0.049552,-392.7641,0.0,
Sex,0.088169,7.968,0.0,
Diabetes,0.116838,8.6214,0.0,
Education,0.041313,3.2998,0.000969,
Live Alone,0.172976,11.4751,0.0,
House Condition,0.019549,9.1167,0.0,
Hypertension,0.058939,5.6662,0.0,
Heart Disease,0.149358,10.2895,0.0,
Alcohol consumption,0.058988,5.0231,0.000001,


### All LAC

In [73]:
#modelRidge_sum_all[0].to_csv('./res_files/Cross_cog/Ridge_All.csv')
modelRidge_sum_all[0]

,Estimate,t value,p value,value
_intercept,1.364116,18.8765,0.0,
Age,-0.036876,-347.2172,0.0,
Sex,0.079425,8.5688,0.0,
Diabetes,0.101981,8.8319,0.0,
Education,0.019564,2.0948,0.036194,
Live Alone,0.000439,0.0443,0.964655,
House Condition,0.020594,12.6262,0.0,
Hypertension,0.059583,6.7202,0.0,
Heart Disease,0.162963,13.1667,0.0,
Alcohol consumption,0.122735,15.3798,0.0,


In [74]:
#modelRidge_sum_all[0].to_csv('./res_files/Cross_cog/Ridge_All.csv')
modelRidge_sum_all2[0]

,Estimate,t value,p value,value
_intercept,2.596479,29.2439,0.0,
Age,-0.048641,-396.5553,0.0,
Sex,0.082511,7.6818,0.0,
Diabetes,0.119917,9.1271,0.0,
Education,0.045347,3.8163,0.000136,
Live Alone,0.09041,7.2335,0.0,
House Condition,0.017854,8.733,0.0,
Hypertension,0.060388,6.0182,0.0,
Heart Disease,0.155054,11.3016,0.0,
Alcohol consumption,0.058317,6.066,0.0,


### Brazil

In [75]:
#modelRidge_sum_all[0].to_csv('./res_files/Cross_cog/Ridge_All.csv')
modelRidge_sum_br[0]

,Estimate,t value,p value,value
_intercept,-0.815307,-6.6861,0.0,
Age,-0.010021,-51.4258,0.0,
Sex,0.026028,1.5293,0.126226,
Diabetes,0.119388,5.5141,0.0,
Education,0.060849,4.1331,0.000036,
Live Alone,-0.057578,-2.473,0.013419,
House Condition,0.009947,3.6228,0.000293,
Hypertension,0.068636,4.1023,0.000041,
Heart Disease,0.207526,8.0602,0.0,
Alcohol consumption,0.070901,2.5318,0.011367,


### MSE and RMSE

In [76]:
mse_rmse_dict = {}

mse_rmse_dict['MSE_HIC'] = modelRidge_sum_chile_uruguay[4]
mse_rmse_dict['MSE_LMIC'] = modelRidge_sum_ecuador_colombia[4]
mse_rmse_dict['MSE_LMIC_withBR'] = modelRidge_sum_ecu_col_br[4]
mse_rmse_dict['MSE_ALL'] = modelRidge_sum_all[4]
mse_rmse_dict['MSE_ALL2'] = modelRidge_sum_all2[4]
mse_rmse_dict['MSE_BR'] = modelRidge_sum_br[4]


mse_rmse_pd = pd.DataFrame(mse_rmse_dict, index = ['MSE']).T

mse_rmse_pd['RMSE'] = [modelRidge_sum_chile_uruguay[5],
                       modelRidge_sum_ecuador_colombia[5],
                       modelRidge_sum_ecu_col_br[5],
                       modelRidge_sum_all[5],
                       modelRidge_sum_all2[5],
                       modelRidge_sum_br[5]]

mse_rmse_pd

,MSE,RMSE
MSE_HIC,0.745125,0.863206
MSE_LMIC,0.657565,0.810904
MSE_LMIC_withBR,0.665799,0.815965
MSE_ALL,0.673785,0.820844
MSE_ALL2,0.667507,0.817011
MSE_BR,0.537404,0.733079


## Model parameters

In [151]:
parameter_pd = pd.DataFrame(parameter_dict).T
parameter_pd.reset_index(drop=True, inplace=True)
parameter_pd[''] =["HIC", "LMIC", "ALL"]
parameter_pd.set_index('', inplace=True)
parameter_pd#.to_excel('Results/cross_sectional/MMSE_hyperparameters.xlsx')

ValueError: Length of values (3) does not match length of index (6)

## Confidence intervals

In [1]:
conf_interval_dict = {}


conf_interval_dict['HIC'] = [conf_interval(X_chile_uruguay_n_IMC_ACV)]
conf_interval_dict['LMIC'] = [conf_interval(X_ecu_col_br_n_IMC_ACV)]
conf_interval_dict['ALL'] = [conf_interval(X_all_n_IMC_ACV)]

conf_interval_pd = pd.DataFrame(conf_interval_dict, index = ['conf interval']).T
conf_interval_pd#.to_excel('Results/cross_sectional/MMSE_conf_interval.xlsx')

NameError: name 'conf_interval' is not defined